## 1/ Merging datasets: adding new features

In [1]:
import pandas as pd
from sklearn.impute import SimpleImputer

In [2]:
df = pd.read_csv('./Datasets/train-data.csv', delimiter=';')

In [3]:
#Change the names of countries that are in the training data to real names (this will be applied to all datasets)
country_mapping = {'DE': "Germany",
'CN': "China",
'GB': "United Kingdom",
'AU': "Australia",
'ES': "Spain",
'NL': "Netherlands",
'US': "United States",
'DK': "Denmark",
'BE': "Belgium",
'FR': "France",
'IT': "Italy",
'PL': "Poland",
'SE': "Sweden",
'TW': "Taiwan",
'JP': "Japan",
'HK': "Hong Kong",
'KR': "South Korea",
'PH': "Philippines",
'MY': "Malaysia",
'SG': "Singapore",
'TH': "Thailand",
'ID': "Indonesia",
'FJ': "Fiji",
'VN': "Vietnam",
'BN': "Brunei",
'NZ': "New Zealand",
'MM': "Myanmar",
'KH': "Cambodia",
'MN': "Mongolia",
'LA': "Laos",
'PG': "Papua New Guinea"}
for original, replacement in country_mapping.items():
    df['Country'] = df['Country'].replace(original, replacement)

In [4]:
######## LPIextend

In [5]:
lpiextend = pd.read_csv('./Datasets/Other datasets/LPIextend.csv', delimiter=',')

In [6]:
cleaned_countries = [country.split(',')[0].strip() for country in lpiextend["Country"].unique()]

In [7]:
country_mapping = dict(zip(lpiextend["Country"].unique(), cleaned_countries))
country_mapping["Taiwan, Chnia"] = "China" #Particular Case

In [8]:
for original, replacement in country_mapping.items():
    lpiextend['Country'] = lpiextend['Country'].replace(original, replacement)

In [9]:
filtered_lpiextend = lpiextend[lpiextend["Country"].isin(df["Country"].unique())] #We choose the countries that are only in the training data

In [10]:
lpiextend2 = filtered_lpiextend[['population (2023)', 'Country', 'growthRate', 'landAreaKm']]  # We choose relevant features

In [11]:
lpiextend2 = lpiextend2[lpiextend2['Country'] != 'Taiwan']  #Cas particulier

In [12]:
######## Economic

In [13]:
economic = pd.read_csv('./Datasets/Other datasets/worldbank_economic_data.csv', delimiter=',')

In [14]:
cleaned_countries2 = [country.split(',')[0].strip() for country in economic["Country"].unique()]

In [15]:
country_mapping2 = dict(zip(economic["Country"].unique(), cleaned_countries2))
country_mapping2["Taiwan, Chnia"] = "China"

In [16]:
for original, replacement in country_mapping2.items():
    economic['Country'] = economic['Country'].replace(original, replacement)

In [17]:
filtered_economic = economic[economic["Country"].isin(df["Country"].unique())]

In [18]:
economic2 = filtered_economic[['Country', 'Year', 'GDP (current US$)', 'Gross capital formation (annual % growth)', 'Manufacturing, value added (annual % growth)']]

In [19]:
########## Imputing missing values in the subsets

In [21]:
imputer = SimpleImputer(strategy='mean')
columns_to_impute = ['GDP (current US$)', 'Gross capital formation (annual % growth)', 'Manufacturing, value added (annual % growth)']
economic2[columns_to_impute] = imputer.fit_transform(economic2[columns_to_impute])

/tmp/ipykernel_547/3090488186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  economic2[columns_to_impute] = imputer.fit_transform(economic2[columns_to_impute])


In [22]:
########## Merging

In [24]:
economic = economic2
lpiextend = lpiextend2

In [25]:
##### First, we need to create new columns in the training dataframe that contains the Year separately. (Year is the column used for merging)

In [26]:
df['Year'] = pd.to_numeric(df['Date'].str.extract(r'(\d{4})')[0], errors='coerce')

In [27]:
merged_df = pd.merge(df, economic, on=['Country', 'Year'], how='left')

In [28]:
merged_df2 = pd.merge(merged_df, lpiextend, on='Country', how='left')

In [29]:
merged_df2.drop(columns='Year', inplace=True)

In [31]:
#### merging with GSCPI

In [32]:
data_GSCPI=pd.read_csv('./Datasets/Other datasets/GSCPI_data.csv', delimiter=',')
data_GSCPI=data_GSCPI[['Year-Month','GSCPI']]

In [33]:
# Split 'Year-Month' column into 'Year' and 'Month'
data_GSCPI[['Year', 'Month']] = data_GSCPI['Year-Month'].str.split('-', expand=True)

# Convert 'Year' and 'Month' columns to integers
data_GSCPI['Year'] = data_GSCPI['Year'].astype(int)
data_GSCPI['Month'] = data_GSCPI['Month'].astype(int)

In [34]:
df2=data_GSCPI
# Créez une nouvelle colonne "groupe" basée sur les conditions spécifiées
df2['groupe'] = pd.cut(df2['Month'], bins=[0, 4, 8, 12], labels=['1', '5', '9'], include_lowest=True)

In [35]:
# Groupez par année et groupe, puis effectuez l'agrégation souhaitée (par exemple, la somme)
resultat = df2.groupby(['Year', 'groupe'])['GSCPI'].mean()

In [36]:
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "1"), 'GSCPI'] = '1.747447'
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "5"), 'GSCPI'] = '2.221625'
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "9"), 'GSCPI'] = '0.784687'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "1"), 'GSCPI'] = '2.030159'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "5"), 'GSCPI'] = '2.957795'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "9"), 'GSCPI'] = '3.897540'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "1"), 'GSCPI'] = '3.117173'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "5"), 'GSCPI'] = '2.052432'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "9"), 'GSCPI'] = '1.111621'
df2.loc[(df2['Year'] == 2023) & (df2['groupe'] == "1"), 'GSCPI'] = '-0.430277'
df2.loc[(df2['Year'] == 2023) & (df2['groupe'] == "5"), 'GSCPI'] = '-1.153880'

In [37]:
df2['groupe'] = df2['groupe'].astype(int)
df2['GSCPI'] = df2['GSCPI'].astype(float)

In [38]:
data=merged_df2

In [39]:
# Adding 'year, starting month and end month' for each row
# Map month names to numeric values

month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

# Extract year and month from 'Date' column in df
data[['Start_Month', 'End_Month', 'Year']] = data['Date'].astype('str').str.extract(r'(\w+)-(\w+) (\d+)', expand=True)

# Map month names to numeric values
data['Start_Month'] = data['Start_Month'].map(month_mapping)
data['End_Month'] = data['End_Month'].map(month_mapping)

# Convert year columns to integers
data['Year'] = data['Year'].astype(int)

In [40]:
df2=df2.drop(['Year-Month','Month'],axis=1)

In [41]:
df_merged_test=pd.merge(data,df2, left_on=['Year','Start_Month'], right_on=['Year', 'groupe'],how='left')
df_merged_test.head()

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,Start_Month,End_Month,Year,GSCPI,groupe
0,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,3.50,5.070000,8.329463e+07,-0.0009,349390.0,5,8,2021,2.957795,5
1,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,3.50,5.070000,8.329463e+07,-0.0009,349390.0,5,8,2021,2.957795,5
2,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,3.50,5.070000,8.329463e+07,-0.0009,349390.0,5,8,2021,2.957795,5
3,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,3.50,5.070000,8.329463e+07,-0.0009,349390.0,5,8,2021,2.957795,5
4,469488,224631,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,...,3.87,1.528281,1.425671e+09,-0.0002,9424702.9,1,4,2021,2.030159,1


In [42]:
df_merged_test.drop_duplicates(inplace=True)

In [44]:
df_merged_test.drop(['Start_Month','End_Month','Year','groupe'],axis=1,inplace=True)

In [45]:
df_merged_test

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Month 2,Month 3,Month 4,GDP (current US$),Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,GSCPI
0,645874,156160,EUROPE,Germany,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,...,0,0,0,4.259935e+12,3.500000,5.070000,8.329463e+07,-0.0009,349390.0,2.957795
4,469488,224631,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,...,1,0,0,1.782046e+13,3.870000,1.528281,1.425671e+09,-0.0002,9424702.9,2.030159
8,348904,104047,EUROPE,United Kingdom,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,...,2,0,0,3.122480e+12,13.980000,10.080000,6.773680e+07,0.0034,241930.0,2.030159
12,1725822,11823,EAJP,Australia,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,...,0,0,0,2.606698e+12,3.085818,1.528281,2.643911e+07,0.0100,7692020.0,-0.430277
16,404781,159924,EUROPE,Spain,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,...,0,0,0,1.427381e+12,5.860000,8.910000,4.751963e+07,-0.0008,499556.6,2.030159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12340940,1445304,221019,CHINA,China,CN_DC_SHD,China Operations,China & HK,China,reference-17197,Product Line-2,...,0,0,0,1.796317e+13,3.620000,1.528281,1.425671e+09,-0.0002,9424702.9,2.052432
12340994,738575,4004,EAJP,Australia,AU_DC_Perth,International Operations,Pacific,Australia,reference-309,Product Line-1,...,1,0,0,1.552703e+12,3.085818,2.240000,2.643911e+07,0.0100,7692020.0,3.897540
12341002,68026,68026,NAM,United States,US_DC_Mechanicsburg,North America Operations,US,USA,reference-7393,Product Line-4,...,1,0,0,2.106047e+13,-3.800000,-4.600000,3.399966e+08,0.0050,9147420.0,0.784687
12341006,1841243,127244,EUROPE,Italy,IT_DC_Venaria,Europe Operations,Italy,Italy,reference-8620,Product Line-4,...,1,0,0,2.606698e+12,3.085818,1.528281,5.887076e+07,-0.0028,295717.0,-0.430277


## 2/ Cleaning the new dataset

In [46]:
#Read the data
df_new=df_merged_test

In [47]:
# Converting months to numerical values
df_new['Month 1'] = df_new['Month 1'].str.replace(' ', '').astype(float)
df_new['Month 2'] = df_new['Month 2'].str.replace(' ', '').astype(float)
df_new['Month 3'] = df_new['Month 3'].str.replace(' ', '').astype(float)
df_new['Month 4'] = df_new['Month 4'].str.replace(' ', '').astype(float)

In [48]:
# Replacing the NaN value in 'Product Life cycel status' column which means Activelifecycle)
df_new['Product Life cycel status'].fillna('active', inplace=True)

### Replacing Missing values

#### missing values in Month 1 of the original dataset

In [49]:
may_jul_2023_data = df_new[df_new['Date'] == 'jan-apr 2023'][['id_product','Month 4']]

# Renommer la colonne "Month 4" pour éviter un conflit lors de la fusion
may_jul_2023_data = may_jul_2023_data.rename(columns={'Month 4': 'New_Month_1'})

# Fusionner les deux DataFrames sur la colonne "id_product"
df_new = pd.merge(df_new, may_jul_2023_data, on=['id_product'], how='left')

# Remplacer les valeurs NaN dans "Month 1" par les valeurs correspondantes dans "New_Month_1"
df_new['Month 1'].fillna(df_new['New_Month_1'], inplace=True)

# Supprimer la colonne temporaire "New_Month_1"
df_new.drop(columns=['New_Month_1'], inplace=True)

In [52]:
df_new['Month 1'].fillna(df_new[['Month 2','Month 3']].mean(axis=1), inplace=True)

#### missing values in the new added features 

In [66]:
df_new.isna().sum()

index                                               0
id_product                                          0
Region                                              0
Country                                             0
Site                                                0
Operations                                          0
Zone                                                0
Cluster                                             0
Reference proxy                                     0
Product  Line proxy                                 0
Division proxy                                      0
Customer Persona proxy                              0
Strategic Product Family proxy                      0
Product Life cycel status                           0
Date                                                0
Month 1                                             0
Month 2                                             0
Month 3                                             0
Month 4                     

In [64]:
gdp= df_new[['GDP (current US$)']].mean()
gdp

GDP (current US$)    6.405325e+12
dtype: float64

In [65]:
df_new['GDP (current US$)'].fillna(6.405325e+12,inplace=True)

In [68]:
gc= df_new[['Gross capital formation (annual % growth)']].mean()
gc

Gross capital formation (annual % growth)    4.167608
dtype: float64

In [69]:
df_new['Gross capital formation (annual % growth)'].fillna(4.167608, inplace=True)

In [70]:
mv= df_new[['Manufacturing, value added (annual % growth)']].mean()
mv

Manufacturing, value added (annual % growth)    2.325467
dtype: float64

In [71]:
df_new['Manufacturing, value added (annual % growth)'].fillna(2.325467, inplace=True)

In [74]:
pop = df_new[['population (2023)']].mean()
gRate = df_new[['growthRate']].mean()
lKm = df_new[['landAreaKm']].mean()

In [75]:
pop

population (2023)    2.272018e+08
dtype: float64

In [76]:
gRate

growthRate    0.004524
dtype: float64

In [77]:
lKm

landAreaKm    3.918378e+06
dtype: float64

In [78]:
df_new['population (2023)'].fillna(2.272018e+08, inplace=True)
df_new['growthRate'].fillna(0.004524, inplace=True)
df_new['landAreaKm'].fillna(3.918378e+06, inplace=True)

In [79]:
df_new.isna().sum()

index                                           0
id_product                                      0
Region                                          0
Country                                         0
Site                                            0
Operations                                      0
Zone                                            0
Cluster                                         0
Reference proxy                                 0
Product  Line proxy                             0
Division proxy                                  0
Customer Persona proxy                          0
Strategic Product Family proxy                  0
Product Life cycel status                       0
Date                                            0
Month 1                                         0
Month 2                                         0
Month 3                                         0
Month 4                                         0
GDP (current US$)                               0


In [82]:
df_new.drop(['index'],axis=1,inplace=True)

#### Removing highly-correlated features ('Region','Country','Site','Operations','Zone' ) with 'Cluster'

In [89]:
df_new.drop(['Region','Country','Site','Operations','Zone'],axis=1,inplace=True)

In [90]:
df_new

,id_product,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4,GDP (current US$),Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,GSCPI
0,156160,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,active,may-aug 2021,0.0,0.0,0.0,0.0,4.259935e+12,3.500000,5.070000,8.329463e+07,-0.0009,349390.0,2.957795
1,224631,China,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,active,jan-apr 2021,0.0,1.0,0.0,0.0,1.782046e+13,3.870000,1.528281,1.425671e+09,-0.0002,9424702.9,2.030159
2,104047,United Kingdom,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,Strategic Product Family-7,active,jan-apr 2021,0.0,2.0,0.0,0.0,3.122480e+12,13.980000,10.080000,6.773680e+07,0.0034,241930.0,2.030159
3,11823,Australia,reference-672,Product Line-1,Division-1,Customer Segmentation-11,Strategic Product Family-3,active,jan-apr 2023,0.0,0.0,0.0,0.0,2.606698e+12,3.085818,1.528281,2.643911e+07,0.0100,7692020.0,-0.430277
4,159924,Spain,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,Strategic Product Family-7,active,jan-apr 2021,0.0,0.0,0.0,0.0,1.427381e+12,5.860000,8.910000,4.751963e+07,-0.0008,499556.6,2.030159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1762965,221019,China,reference-17197,Product Line-2,Division-2,Customer Segmentation-28,Strategic Product Family-5,active,may-aug 2022,0.0,0.0,0.0,0.0,1.796317e+13,3.620000,1.528281,1.425671e+09,-0.0002,9424702.9,2.052432
1762966,4004,Australia,reference-309,Product Line-1,Division-1,Customer Segmentation-8,Strategic Product Family-3,EOL,sep-dec 2021,0.0,1.0,0.0,0.0,1.552703e+12,3.085818,2.240000,2.643911e+07,0.0100,7692020.0,3.897540
1762967,68026,USA,reference-7393,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-9,active,sep-dec 2020,18.0,1.0,0.0,0.0,2.106047e+13,-3.800000,-4.600000,3.399966e+08,0.0050,9147420.0,0.784687
1762968,127244,Italy,reference-8620,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-12,active,jan-apr 2023,0.0,1.0,0.0,0.0,2.606698e+12,3.085818,1.528281,5.887076e+07,-0.0028,295717.0,-0.430277


In [91]:
df_new.to_csv("train_preprocessing_clean.csv", index=False, sep=';')

## Test Set Preprocesing

In [ ]:
#################### Test Set preprocessing #####################

In [92]:
df = pd.read_csv('./Datasets/X_test.csv', delimiter=';')

In [94]:
#Change the names of countries that are in the training data to real names (this will be applied to all datasets)
country_mapping = {'DE': "Germany",
'CN': "China",
'GB': "United Kingdom",
'AU': "Australia",
'ES': "Spain",
'NL': "Netherlands",
'US': "United States",
'DK': "Denmark",
'BE': "Belgium",
'FR': "France",
'IT': "Italy",
'PL': "Poland",
'SE': "Sweden",
'TW': "Taiwan",
'JP': "Japan",
'HK': "Hong Kong",
'KR': "South Korea",
'PH': "Philippines",
'MY': "Malaysia",
'SG': "Singapore",
'TH': "Thailand",
'ID': "Indonesia",
'FJ': "Fiji",
'VN': "Vietnam",
'BN': "Brunei",
'NZ': "New Zealand",
'MM': "Myanmar",
'KH': "Cambodia",
'MN': "Mongolia",
'LA': "Laos",
'PG': "Papua New Guinea"}
for original, replacement in country_mapping.items():
    df['Country'] = df['Country'].replace(original, replacement)

In [95]:
######## LPIextend

In [96]:
lpiextend = pd.read_csv('./Datasets/Other datasets/LPIextend.csv', delimiter=',')

In [97]:
cleaned_countries = [country.split(',')[0].strip() for country in lpiextend["Country"].unique()]

In [98]:
country_mapping = dict(zip(lpiextend["Country"].unique(), cleaned_countries))
country_mapping["Taiwan, Chnia"] = "China" #Particular Case

In [99]:
for original, replacement in country_mapping.items():
    lpiextend['Country'] = lpiextend['Country'].replace(original, replacement)

In [100]:
filtered_lpiextend = lpiextend[lpiextend["Country"].isin(df["Country"].unique())] #We choose the countries that are only in the training data

In [101]:
lpiextend2 = filtered_lpiextend[['population (2023)', 'Country', 'growthRate', 'landAreaKm']]  # We choose relevant features

In [102]:
lpiextend2 = lpiextend2[lpiextend2['Country'] != 'Taiwan']  #Cas particulier

In [103]:
######## Economic

In [104]:
economic = pd.read_csv('./Datasets/Other datasets/worldbank_economic_data.csv', delimiter=',')

In [105]:
cleaned_countries2 = [country.split(',')[0].strip() for country in economic["Country"].unique()]

In [106]:
country_mapping2 = dict(zip(economic["Country"].unique(), cleaned_countries2))
country_mapping2["Taiwan, Chnia"] = "China"

In [107]:
for original, replacement in country_mapping2.items():
    economic['Country'] = economic['Country'].replace(original, replacement)

In [108]:
filtered_economic = economic[economic["Country"].isin(df["Country"].unique())]

In [109]:
economic2 = filtered_economic[['Country', 'Year', 'GDP (current US$)', 'Gross capital formation (annual % growth)', 'Manufacturing, value added (annual % growth)']]

In [110]:
########## Imputing missing values in the subsets

In [111]:
imputer = SimpleImputer(strategy='mean')
columns_to_impute = ['GDP (current US$)', 'Gross capital formation (annual % growth)', 'Manufacturing, value added (annual % growth)']
economic2[columns_to_impute] = imputer.fit_transform(economic2[columns_to_impute])

/tmp/ipykernel_547/3090488186.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  economic2[columns_to_impute] = imputer.fit_transform(economic2[columns_to_impute])


In [112]:
########## Merging

In [113]:
economic = economic2
lpiextend = lpiextend2

In [114]:
##### First, we need to create new columns in the training dataframe that contains the Year separately. (Year is the column used for merging)

In [115]:
df['Year'] = pd.to_numeric(df['Date'].str.extract(r'(\d{4})')[0], errors='coerce')

In [116]:
merged_df = pd.merge(df, economic, on=['Country', 'Year'], how='left')

In [117]:
merged_df2 = pd.merge(merged_df, lpiextend, on='Country', how='left')

In [118]:
merged_df2.drop(columns='Year', inplace=True)

In [119]:
#### merging with GSCPI

In [120]:
data_GSCPI=pd.read_csv('./Datasets/Other datasets/GSCPI_data.csv', delimiter=',')
data_GSCPI=data_GSCPI[['Year-Month','GSCPI']]

In [121]:
# Split 'Year-Month' column into 'Year' and 'Month'
data_GSCPI[['Year', 'Month']] = data_GSCPI['Year-Month'].str.split('-', expand=True)

# Convert 'Year' and 'Month' columns to integers
data_GSCPI['Year'] = data_GSCPI['Year'].astype(int)
data_GSCPI['Month'] = data_GSCPI['Month'].astype(int)

In [122]:
df2=data_GSCPI
# Créez une nouvelle colonne "groupe" basée sur les conditions spécifiées
df2['groupe'] = pd.cut(df2['Month'], bins=[0, 4, 8, 12], labels=['1', '5', '9'], include_lowest=True)

In [123]:
# Groupez par année et groupe, puis effectuez l'agrégation souhaitée (par exemple, la somme)
resultat = df2.groupby(['Year', 'groupe'])['GSCPI'].mean()

In [124]:
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "1"), 'GSCPI'] = '1.747447'
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "5"), 'GSCPI'] = '2.221625'
df2.loc[(df2['Year'] == 2020) & (df2['groupe'] == "9"), 'GSCPI'] = '0.784687'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "1"), 'GSCPI'] = '2.030159'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "5"), 'GSCPI'] = '2.957795'
df2.loc[(df2['Year'] == 2021) & (df2['groupe'] == "9"), 'GSCPI'] = '3.897540'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "1"), 'GSCPI'] = '3.117173'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "5"), 'GSCPI'] = '2.052432'
df2.loc[(df2['Year'] == 2022) & (df2['groupe'] == "9"), 'GSCPI'] = '1.111621'
df2.loc[(df2['Year'] == 2023) & (df2['groupe'] == "1"), 'GSCPI'] = '-0.430277'
df2.loc[(df2['Year'] == 2023) & (df2['groupe'] == "5"), 'GSCPI'] = '-1.153880'

In [125]:
df2['groupe'] = df2['groupe'].astype(int)
df2['GSCPI'] = df2['GSCPI'].astype(float)

In [126]:
data=merged_df2

In [127]:
# Adding 'year, starting month and end month' for each row
# Map month names to numeric values

month_mapping = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6,
                 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}

# Extract year and month from 'Date' column in df
data[['Start_Month', 'End_Month', 'Year']] = data['Date'].astype('str').str.extract(r'(\w+)-(\w+) (\d+)', expand=True)

# Map month names to numeric values
data['Start_Month'] = data['Start_Month'].map(month_mapping)
data['End_Month'] = data['End_Month'].map(month_mapping)

# Convert year columns to integers
data['Year'] = data['Year'].astype(int)

In [128]:
df2=df2.drop(['Year-Month','Month'],axis=1)

In [129]:
df_merged_test=pd.merge(data,df2, left_on=['Year','Start_Month'], right_on=['Year', 'groupe'],how='left')
df_merged_test.head()

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,Start_Month,End_Month,Year,GSCPI,groupe
0,490548,834,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,Australia,reference-145,Product Line-1,...,3.085818,2.240000,26439111.0,0.010,7692020.0,5,8,2021,2.957795,5
1,490548,834,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,Australia,reference-145,Product Line-1,...,3.085818,2.240000,26439111.0,0.010,7692020.0,5,8,2021,2.957795,5
2,490548,834,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,Australia,reference-145,Product Line-1,...,3.085818,2.240000,26439111.0,0.010,7692020.0,5,8,2021,2.957795,5
3,490548,834,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,Australia,reference-145,Product Line-1,...,3.085818,2.240000,26439111.0,0.010,7692020.0,5,8,2021,2.957795,5
4,2167957,209101,EUROPE,France,FR_DC_Evreux,France Operations,France,France,reference-13121,Product Line-4,...,3.085818,1.528281,64756584.0,0.002,547557.0,5,7,2023,-1.153880,5


In [130]:
df_merged_test.drop_duplicates(inplace=True)

In [131]:
df_merged_test.drop(['Start_Month','End_Month','Year','groupe'],axis=1,inplace=True)

In [132]:
df_merged_test

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,...,Month 1,Month 2,Month 3,GDP (current US$),Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,GSCPI
0,490548,834,EAJP,Australia,AU_DC_Sydney,International Operations,Pacific,Australia,reference-145,Product Line-1,...,20 435,610,4 270,1.552703e+12,3.085818,2.240000,2.643911e+07,0.0100,7692020.0,2.957795
4,2167957,209101,EUROPE,France,FR_DC_Evreux,France Operations,France,France,reference-13121,Product Line-4,...,NaN,0,0,2.606698e+12,3.085818,1.528281,6.475658e+07,0.0020,547557.0,-1.153880
7,1999013,40157,EAJP,New Zealand,AU_DC_ALD,International Operations,Pacific,New Zealand,reference-3409,Product Line-3,...,NaN,0,4,2.606698e+12,3.085818,1.528281,5.228100e+06,0.0083,263310.0,-1.153880
10,469221,224364,CHINA,China,CN_DC_Shanghai,China Operations,China & HK,China,reference-19450,Product Line-4,...,7,150,136,1.782046e+13,3.870000,1.528281,1.425671e+09,-0.0002,9424702.9,2.030159
14,1171716,192288,EUROPE,Belgium,NL_DC_HLD,Europe Operations,BeNe,Belgium,reference-3686,Product Line-3,...,0,0,1,5.786041e+11,1.580000,0.200000,1.168614e+07,0.0026,30280.0,3.117173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713960,849490,114919,EUROPE,Italy,IT_DC_Venaria,Europe Operations,Italy,Italy,reference-6671,Product Line-4,...,0,2,0,2.114356e+12,21.210000,14.070000,5.887076e+07,-0.0028,295717.0,3.897540
1713964,1199858,220430,CHINA,China,CN_MF_SSPA,China Operations,China & HK,China,reference-19181,Product Line-5,...,0,0,0,1.796317e+13,3.620000,1.528281,1.425671e+09,-0.0002,9424702.9,3.117173
1713968,1007643,28215,EAJP,Australia,AU_DC_Brisbane,International Operations,Pacific,Australia,reference-1966,Product Line-1,...,0,0,0,1.675419e+12,3.085818,2.540000,2.643911e+07,0.0100,7692020.0,3.117173
1713972,1647728,178586,EUROPE,Poland,HU_DC_CEELog_Budapest,Europe Operations,CEEI,Middle Eastern Europe,reference-7364,Product Line-4,...,2,0,0,6.881766e+11,16.810000,7.220000,4.102607e+07,0.0293,306130.0,1.111621


## 2/ Cleaning the new dataset

In [136]:
#Read the data
df_new=df_merged_test

In [140]:
# Converting months to numerical values
df_new['Month 1'] = df_new['Month 1'].astype(str).str.replace(' ', '').astype(float)
df_new['Month 2'] = df_new['Month 2'].astype(str).str.replace(' ', '').astype(float)
df_new['Month 3'] = df_new['Month 3'].astype(str).str.replace(' ', '').astype(float)

In [141]:
# Replacing the NaN value in 'Product Life cycel status' column which means Activelifecycle)
df_new['Product Life cycel status'].fillna('active', inplace=True)

### Replacing Missing values

#### missing values in Month 1 of the original dataset

In [143]:
may_jul_2023_data = df_new[df_new['Date'] == 'jan-apr 2023'][['id_product','Month 3']]

# Renommer la colonne "Month 4" pour éviter un conflit lors de la fusion
may_jul_2023_data = may_jul_2023_data.rename(columns={'Month 3': 'New_Month_1'})

# Fusionner les deux DataFrames sur la colonne "id_product"
df_new = pd.merge(df_new, may_jul_2023_data, on=['id_product'], how='left')

# Remplacer les valeurs NaN dans "Month 1" par les valeurs correspondantes dans "New_Month_1"
df_new['Month 1'].fillna(df_new['New_Month_1'], inplace=True)

# Supprimer la colonne temporaire "New_Month_1"
df_new.drop(columns=['New_Month_1'], inplace=True)

In [144]:
df_new['Month 1'].fillna(df_new[['Month 2','Month 3']].mean(axis=1), inplace=True)

#### missing values in the new added features 

In [145]:
df_new.isna().sum()

index                                              0
id_product                                         0
Region                                             0
Country                                            0
Site                                               0
Operations                                         0
Zone                                               0
Cluster                                            0
Reference proxy                                    0
Product  Line proxy                                0
Division proxy                                     0
Customer Persona proxy                             0
Strategic Product Family proxy                     0
Product Life cycel status                          0
Date                                               0
Month 1                                            0
Month 2                                            0
Month 3                                            0
GDP (current US$)                             

In [146]:
gdp= df_new[['GDP (current US$)']].mean()
gdp

GDP (current US$)    6.400696e+12
dtype: float64

In [147]:
df_new['GDP (current US$)'].fillna(6.400696e+12,inplace=True)

In [148]:
gc= df_new[['Gross capital formation (annual % growth)']].mean()
gc

Gross capital formation (annual % growth)    4.171108
dtype: float64

In [149]:
df_new['Gross capital formation (annual % growth)'].fillna(4.171108, inplace=True)

In [150]:
mv= df_new[['Manufacturing, value added (annual % growth)']].mean()
mv

Manufacturing, value added (annual % growth)    2.324732
dtype: float64

In [151]:
df_new['Manufacturing, value added (annual % growth)'].fillna(2.324732, inplace=True)

In [152]:
pop = df_new[['population (2023)']].mean()
gRate = df_new[['growthRate']].mean()
lKm = df_new[['landAreaKm']].mean()

In [153]:
pop

population (2023)    2.263951e+08
dtype: float64

In [154]:
gRate

growthRate    0.004526
dtype: float64

In [155]:
lKm

landAreaKm    3.914891e+06
dtype: float64

In [156]:
df_new['population (2023)'].fillna(2.263951e+08, inplace=True)
df_new['growthRate'].fillna(0.004526, inplace=True)
df_new['landAreaKm'].fillna(3.914891e+06, inplace=True)

In [157]:
df_new.isna().sum()

index                                           0
id_product                                      0
Region                                          0
Country                                         0
Site                                            0
Operations                                      0
Zone                                            0
Cluster                                         0
Reference proxy                                 0
Product  Line proxy                             0
Division proxy                                  0
Customer Persona proxy                          0
Strategic Product Family proxy                  0
Product Life cycel status                       0
Date                                            0
Month 1                                         0
Month 2                                         0
Month 3                                         0
GDP (current US$)                               0
Gross capital formation (annual % growth)       0


In [158]:
df_new.drop(['index'],axis=1,inplace=True)

#### Removing highly-correlated features ('Region','Country','Site','Operations','Zone' ) with 'Cluster'

In [159]:
df_new.drop(['Region','Country','Site','Operations','Zone'],axis=1,inplace=True)

In [160]:
df_new

,id_product,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,GDP (current US$),Gross capital formation (annual % growth),"Manufacturing, value added (annual % growth)",population (2023),growthRate,landAreaKm,GSCPI
0,834,Australia,reference-145,Product Line-1,Division-1,Customer Segmentation-8,Strategic Product Family-1,EOL,may-aug 2021,20435.0,610.0,4270.0,1.552703e+12,3.085818,2.240000,2.643911e+07,0.0100,7692020.0,2.957795
1,209101,France,reference-13121,Product Line-4,Division-3,Customer Segmentation-66,Strategic Product Family-9,active,may-jul 2023,0.0,0.0,0.0,2.606698e+12,3.085818,1.528281,6.475658e+07,0.0020,547557.0,-1.153880
2,40157,New Zealand,reference-3409,Product Line-3,Division-2,Customer Segmentation-4,Strategic Product Family-7,active,may-jul 2023,2.0,0.0,4.0,2.606698e+12,3.085818,1.528281,5.228100e+06,0.0083,263310.0,-1.153880
3,224364,China,reference-19450,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-13,active,jan-apr 2021,7.0,150.0,136.0,1.782046e+13,3.870000,1.528281,1.425671e+09,-0.0002,9424702.9,2.030159
4,192288,Belgium,reference-3686,Product Line-3,Division-2,Customer Segmentation-4,Strategic Product Family-7,RaMPDOWN,jan-apr 2022,0.0,0.0,1.0,5.786041e+11,1.580000,0.200000,1.168614e+07,0.0026,30280.0,3.117173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440738,114919,Italy,reference-6671,Product Line-4,Division-3,Customer Segmentation-15,Strategic Product Family-9,active,sep-dec 2021,0.0,2.0,0.0,2.114356e+12,21.210000,14.070000,5.887076e+07,-0.0028,295717.0,3.897540
440739,220430,China,reference-19181,Product Line-5,Division-3,Customer Segmentation-13,Strategic Product Family-15,active,jan-apr 2022,0.0,0.0,0.0,1.796317e+13,3.620000,1.528281,1.425671e+09,-0.0002,9424702.9,3.117173
440740,28215,Australia,reference-1966,Product Line-1,Division-1,Customer Segmentation-6,Strategic Product Family-4,active,jan-apr 2022,0.0,0.0,0.0,1.675419e+12,3.085818,2.540000,2.643911e+07,0.0100,7692020.0,3.117173
440741,178586,Middle Eastern Europe,reference-7364,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,active,sep-dec 2022,2.0,0.0,0.0,6.881766e+11,16.810000,7.220000,4.102607e+07,0.0293,306130.0,1.111621


In [161]:
df_new.to_csv("test_preprocessing_clean.csv", index=False, sep=';')